# Import transaction data and aggregate it

 This workflow will demonstrate how to connect SQLite database with python to import and aggregate data

## Problem Statement
Create scripts that will process the CSV files, insert data into table and use that data in next step for performing various aggregations on it. The columns in the CSV files are as follow:

“transactionId”, // a unique transaction identifier <br />
“user”, // a unique user identifier <br />
“datetime”, // a timestamp in ms (javascript format) <br />
“operation”, // a description of the transaction being charged <br />
“quantity”, <br />
“unitPrice” // the revenue of the transaction is quantity * unit_price <br />
We have use SQLite as a databse so the test is self-contained. 

# Expected Result

### Part 1: Create an import script

In the first part, we will create a script that imports the transactions of the CSV files into an SQL table.

- The script should accept the CSV filename.
- It should put the information into the DB, in a table that collects all transactions.
- All data from CSVs imported by the script will be placed in this same table.
- It should handle duplicate rows (i.e. same CSV imported twice by mistake or update existing data by processing a new file).

### Part 2: Create an aggregation script

In the second part, we will create a program that reads from the SQL table from the previous part and aggregates data into two new DB tables that allow to get the following information:

- Aggregated by user / day: Number of operations and revenue per User per Day
- Aggregated by user / hour: Number of operations and revenue per User per Hour

### Getting Started

###### First, Import SQLite3 as a database 

In [128]:
import pandas as pd
import sqlite3
from csv import reader
from csv import DictReader
import datetime

###### Second, create a Database object and connect it.

In [129]:
database = 'hexonet.db'
connection = sqlite3.connect(database)

###### Read CSV file 

In [130]:
df = pd.read_csv('transactions_2020-08-02.csv')

###### Get the details about the data stored in CSV file

In [131]:
print(df)
df.info()

       transactionId     user       datetime    operation  quantity  unitPrice
0              30000  user_57  1596351600956  OPERATION_4         6      47.89
1              30001  user_81  1596351602450  OPERATION_9         3      79.23
2              30002  user_33  1596351603777  OPERATION_8         9      69.88
3              30003  user_37  1596351606674  OPERATION_9         4      62.63
4              30004  user_24  1596351615957  OPERATION_0         5      81.53
...              ...      ...            ...          ...       ...        ...
24995          54994  user_18  1596437988089  OPERATION_9         1      83.06
24996          54996  user_96  1596437989006  OPERATION_6         7      35.93
24997          54997  user_20  1596437990091  OPERATION_7         6      19.13
24998          54998  user_53  1596437995615  OPERATION_0         4      52.71
24999          54999  user_18  1596437998268  OPERATION_1         2      90.36

[25000 rows x 6 columns]
<class 'pandas.core.frame.

###### Create table named 'tran' to store the records fetched from CSV files.

In [124]:
cur = connection.cursor()
sql = '''
CREATE TABLE tran(
transactionId INT, 
user TEXT, 
datetime NUMERIC, 
operation TEXT, 
quantity INT, 
unitPrice NUMERIC,
tranDate TEXT,
fileName TEXT
)'''
cur.execute(sql)
print('Table Created...')

Table Created...


###### Open the CSV file, read the data and insert these records into tran table.

In [139]:
with open('transactions_2020-08-01.csv', 'r') as read_obj:
    csv_dict_reader = DictReader(read_obj)
    for row in csv_dict_reader:
        #s = row['datetime']
        s = 1596330211174
        date = int(s) / 1000.0
        tranDate = datetime.datetime.fromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S.%f')
       #print(row['transactionId'], row['user'],row['datetime'],row['operation'],row['quantity'],row['unitPrice'],tranDate,'transactions_2020-08-02.csv')
        cur.execute("SELECT EXISTS(SELECT 1 FROM tran WHERE transactionId=? LIMIT 1)", (row['transactionId'],))
        record = cur.fetchone()
        if record[0] == 1:
              cur.execute(''' update tran set user = ?, datetime= ?, operation = ?, quantity = ?, unitPrice = ?, tranDate = ?, fileName = ? where transactionId = ?''', (row['user'],row['datetime'],row['operation'],row['quantity'],row['unitPrice'],tranDate,'transactions_2020-08-02.csv',row['transactionId']))
              connection.commit()
              print("Records updated in table")
        else:
              cur.execute('''INSERT INTO tran values (?, ?, ?, ?, ?, ?, ?, ?) ''', ( row['transactionId'], row['user'],row['datetime'],row['operation'],row['quantity'],row['unitPrice'],tranDate,'transactions_2020-08-02.csv'))
              connection.commit()
        print("Records Inserted successfully........")



        
        
        

Records Inserted successfully........


In [79]:
cur.execute("select count(*) from tran")
results = cur.fetchall()
print(results)

[(45000,)]


###### Create new table named 'Agg_User_Per_Day' to store 'Number of operations and revenue per User per Day'

In [125]:
sql = '''
CREATE TABLE Agg_User_Per_Day(
date TEXT,
user TEXT,
No_of_Operations INTEGER,
Revenue NUMERIC
)'''
cur.execute(sql)
print("Table created successfully........")


Table created successfully........


###### Aggregate the data fetched from tran table and insert it into new table.
This will perform following:
- Aggregated by user / day: Number of operations and revenue per User per Day

In [106]:
query = '''
   select substr(tranDate,0,11),user,count(operation),sum(quantity*unitPrice)
   from tran group by substr(tranDate,0,11),user;
        '''
cur.execute(query)
results = cur.fetchall()
#print(results)

cur.executemany('INSERT INTO Agg_User_Per_Day VALUES (?,?,?,?);',results)
#Commit your changes in the database
connection.commit()

###### Fetch the aggregated data from Agg_User_Per_Day

In [108]:
cur.execute("SELECT sum(Revenue) FROM Agg_User_Per_Day ")
results = cur.fetchall()
print(results)

[(12428956.66,)]


###### Create new table named 'Agg_User_Per_Hour' to store 'Number of operations and revenue per User per Hour'

In [119]:
sql = '''
CREATE TABLE Agg_User_Per_Hour(
date TEXT,
hour TEXT,
user TEXT,
No_of_Operations INTEGER,
Revenue NUMERIC
)'''
cur.execute(sql)

###### Aggregate the data fetched from tran table and insert it into new table. This will perform following:
- Aggregated by user / hour: Number of operations and revenue per User per Hour

In [121]:
query = '''
   select substr(tranDate,0,11),substr(tranDate,12,2)as hour,user,count(operation), sum(quantity*unitPrice)
    from tran group by substr(tranDate,0,11),user , substr(tranDate,12,2) ;
        '''
cur.execute(query)
results = cur.fetchall()
#print(results)

cur.executemany('INSERT INTO Agg_User_Per_Hour VALUES (?,?,?,?,?);',results)
# Commit your changes in the database
connection.commit()

###### Fetch the aggregated data from Agg_User_Per_Hour

In [123]:
cur.execute("SELECT sum(Revenue) FROM Agg_User_Per_Hour")
results = cur.fetchall()
print(results)

[(12428956.65999999,)]


# Possible Flaws in this workflow:

1. Further, if we want to prevent same file to get insert multiple times in the database, then we can store the filename and it's upload date-time in separate column and put validation on it.
2. We can utilize available data in more detailed format and generate various reports like:
- Total revenue by each operation
- No.of transactions per day

# Conclusion:
We can analyse this data using various tools like node JS, Python, Tableau, Power BI etc. Additionally, We can visualize this data in Tableau or Power BI to make it easy to understand and presentale.